In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [41]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(7011, 8)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6717, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c":"Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7017, 10)

fin empresas y comienzo de rescate de pacientes


In [5]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1635985, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [6]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1209257, 4)

In [7]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1209257, 13)

In [8]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [9]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [51]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [52]:
#pacientes_empresas22.shape

In [53]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [54]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [55]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [10]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [11]:
Pacientes_Archivo.head(10)

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,birthdate,Unnamed: 8,Unnamed: 9,...,nacimiento,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa
0,340022761,340022761,12880154-5,12880154-5,MIGUEL ANGEL,SAENZ FUENTES,Masculino,24-11-1974,24,11,...,1974-11-24,AS,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA
1,340022761,340022761,12896337-5,12880154-5,MARTA ISABEL,ELGUETA SANDOVAL,Femenino,11-03-1975,11,3,...,1975-03-11,CO,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA
2,340022761,340022761,20381990-0,12880154-5,FERNANDA,SAENZ ELGUETA,Femenino,24-11-1999,24,11,...,1999-11-24,HI,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA
3,340008039,340008039,17326548-4,17326548-4,SERGIO,CARRASCO GONZALEZ,Masculino,15-02-1983,15,2,...,1983-02-15,AS,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA
4,340008039,340008039,16141867-6,17326548-4,MARIA JOSÉ,CONTRERAS PARDO,Femenino,06-05-1985,6,5,...,1985-05-06,CO,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA
5,340008039,340008039,27282604-8,17326548-4,MARIA,CARRASCO CONTRERAS,Femenino,25-05-2020,25,5,...,2020-05-25,HI,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA
6,340008039,340008039,27782634-8,17326548-4,BENJAMIN,CARRASCO CONTRERAS,Masculino,08-05-2022,8,5,...,2022-05-08,HI,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA
7,340020430,340020430,27148926-9,16371184-2,EMILIA,BONGARD LANDABUR,Femenino,04-01-2020,4,1,...,2020-01-04,HI,NaN,NaN,NaN,998845071,Metlife,45018,Servicios Integrales IMC SPA,a1W8c0000083qy1EAA
8,340020352,340020352,12841493-2,10780174-K,PAOLA ALEJANDRA,ASTUDILLO ARIAGA,Femenino,17-11-1974,17,11,...,1974-11-17,CO,NaN,NaN,NaN,988093999,Metlife,45018,Servicios Profesionales Akhand Ltda.,a1W8c000007vDQ9EAM
9,340023115,340023115,19910794-1,19910794-1,JOSIAS SAMUEL,ARANDA PASTEN,Masculino,30-06-1998,30,6,...,1998-06-30,AS,NaN,NaN,NaN,941656343,Metlife,45018,Berliam SPA,a1W8c0000083r5SEAQ


In [12]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [13]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,birthdate,Unnamed: 8,Unnamed: 9,...,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa,rut
0,340022761,340022761,12880154-5,12880154-5,MIGUEL ANGEL,SAENZ FUENTES,Masculino,24-11-1974,24,11,...,AS,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,128801545
1,340022761,340022761,12896337-5,12880154-5,MARTA ISABEL,ELGUETA SANDOVAL,Femenino,11-03-1975,11,3,...,CO,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,128963375
2,340022761,340022761,20381990-0,12880154-5,FERNANDA,SAENZ ELGUETA,Femenino,24-11-1999,24,11,...,HI,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,203819900
3,340008039,340008039,17326548-4,17326548-4,SERGIO,CARRASCO GONZALEZ,Masculino,15-02-1983,15,2,...,AS,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA,173265484
4,340008039,340008039,16141867-6,17326548-4,MARIA JOSÉ,CONTRERAS PARDO,Femenino,06-05-1985,6,5,...,CO,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA,161418676


In [14]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10940\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,birthdate,Unnamed: 8,Unnamed: 9,...,relacionConAsegurado,direccion,ciudad,email,celular,campanaPaciente,fecha_modificacion,empleador,Id Programas Por Empresa,rut
0,340022761,340022761,12880154-5,12880154-5,MIGUEL ANGEL,SAENZ FUENTES,Masculino,24-11-1974,24,11,...,AS,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,128801545
1,340022761,340022761,12896337-5,12880154-5,MARTA ISABEL,ELGUETA SANDOVAL,Femenino,11-03-1975,11,3,...,CO,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,128963375
2,340022761,340022761,20381990-0,12880154-5,FERNANDA,SAENZ ELGUETA,Femenino,24-11-1999,24,11,...,HI,BLANCA FLOR 557,SANTIAGO,NaN,953712170,Metlife,45018,Walmart,a1W8c0000083qvzEAA,203819900
3,340008039,340008039,17326548-4,17326548-4,SERGIO,CARRASCO GONZALEZ,Masculino,15-02-1983,15,2,...,AS,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA,173265484
4,340008039,340008039,16141867-6,17326548-4,MARIA JOSÉ,CONTRERAS PARDO,Femenino,06-05-1985,6,5,...,CO,NaN,NaN,NaN,NaN,Metlife,45018,Merck Sharp & Dohme (I.A) LLC Agencia En.,a1W8c0000083ip3EAA,161418676


In [15]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
# Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10940\1486029825.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_10940\1486029825.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [16]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(10903, 32)

In [17]:
Pacientes_con_id.head()

,poliza,Columna1,rut1,rutAsegurado,nombres,apellidos,sexo,birthdate,Unnamed: 8,Unnamed: 9,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340022761,340022761,12880154-5,12880154-5,Miguel Angel,Saenz Fuentes,Masculino,24-11-1974,24,11,...,True,0018c00002kAx9xAAC,128801545,Miguel Angel Saenz Fuentes,RUT,None,None,None,Hombre,953712170
1,340022761,340022761,12896337-5,12880154-5,Marta Isabel,Elgueta Sandoval,Femenino,11-03-1975,11,3,...,True,0018c00002kAx9yAAC,128963375,Marta Isabel Elgueta Sandoval,RUT,None,None,None,Mujer,953712170
2,340022761,340022761,20381990-0,12880154-5,Fernanda,Saenz Elgueta,Femenino,24-11-1999,24,11,...,True,0018c00002kAx9zAAC,203819900,Fernanda Saenz Elgueta,RUT,None,None,None,Mujer,953712170
3,340008039,340008039,17326548-4,17326548-4,Sergio,Carrasco Gonzalez,Masculino,15-02-1983,15,2,...,True,0018c00002cWWZlAAO,173265484,Carrasco Gonzalez Sergio Antonio,RUT,None,None,None,Hombre,None
4,340008039,340008039,16141867-6,17326548-4,Maria José,Contreras Pardo,Femenino,06-05-1985,6,5,...,True,0018c00002cWurXAAS,161418676,Maria José Contreras Pardo,RUT,None,None,None,Mujer,None


In [18]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.title().replace("Nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingCity,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
8126,False,José,Lagunas Fernández,212859613,,2003-02-21,Hombre,NaN,NaN,,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
8127,False,Krishna,Olivares Cisternas,222590884,,2006-11-14,Mujer,NaN,NaN,,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
8129,False,Ruth,Lagunas Fernández,125717012,,1974-02-05,Mujer,NaN,NaN,,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
8130,False,Belen,Gallardo Lagunas,234650882,,2010-10-29,Mujer,NaN,NaN,,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
8135,False,Cristian Javier,Tapia Lorca,180345760,,1991-10-06,Hombre,NaN,NaN,,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [19]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [20]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [21]:
campaña_seleccionada.shape

(739382, 13)

In [22]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [23]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )

In [24]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
